Próba implementacji metod zastosowanych przez dr. Kontka w artykule 

["**Weryfikacja wyniku drugiej tury wyborów prezydenckich w Polsce w 2025 roku: Przeliczenie głosów z użyciem przestrzennie grupowanej metody MAD**"](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=5296441)

dla wyników wyborów prezydenckich z 2020 roku

In [1]:
import pandas as pd

from utilities import presidential_data, clustering, methods

### load presidential both rouds - official PKW data

In [2]:
year = "2020"
df_2020_r1 = presidential_data.get_df(year, "1")
df_2020_r2 = presidential_data.get_df(year, "2")

### IMPLEMENTACJA METOD ZAPROPONOWANYCH PRZEZ DR KONTKA

2.3. Wykrywanie wartości odstających

Główna innowacja analityczna niniejszego badania polega na oszacowaniu potencjalnego wpływu
anormalnych komisji wyborczych na poziomie ogólnokrajowym. Aby to osiągnąć, w pierwszej
kolejności zidentyfikowano wartości odstające w czterech kategoriach nieprawidłowości:

In [3]:
cand_A = "trzaskowski"
cand_B = "duda"

df_2020 = presidential_data.join_both_rounds(cand_A, cand_B, df_2020_r1, df_2020_r2)

In [4]:
# grupowanie geograficzne metodą p. Białka
df_2020 = clustering.add_bialek_postal_buckets(df_2020)

### 1. pop_outlier: Nadmierne poparcie dla kandydata A (względem mediany w ramach lokalnej grupy)

In [5]:
print("Liczba komisji: ", len(df_2020)) # bez zagranicy i statków
df = df_2020.copy()

Liczba komisji:  26215


In [6]:
df = methods.add_anomaly_1(df, cand_A, cand_B, new_col_name="k_score_1")

In [7]:
methods.check_k_thresholds(df, cand_A, cand_B, "_k_score_1")

k > 2.0
trzaskowski: 3430
duda: 2893
---
k > 2.5
trzaskowski: 2541
duda: 2018
---
k > 3.0
trzaskowski: 1917
duda: 1434
---


Wyniki:

Dla k=2, takich komisji, w których "za duże" poparcie ma Duda jest 2893 a Trzaskowski 3430 (vs 3750 i 4551 w 2025)

### 2. growth_outlier: Nadmierny względny wzrost poparcia dla kandydata A między pierwszą a drugą turą, w porównaniu do odpowiedniego wzrostu poparcia dla kandydata B w tej samej grupie lokalnej;

In [8]:
df = methods.add_anomaly_2(df, cand_A, cand_B, new_col_name="k_score_2")

In [9]:
df.head()

,teryt_gmina,polling_station_id,trzaskowski_r1,duda_r1,postal_code,trzaskowski_r2,duda_r2,postal_clean,bucket,trzaskowski_k_score_1,duda_k_score_1,trzaskowski_k_score_2,duda_k_score_2
0,20101,1,338,367,59-700,512,480,59700,1378,0.306122,3.025,-1.967424,1.967424
1,20101,2,365,311,59-700,536,362,59700,1378,0.551020,0.075,-0.700443,0.700443
2,20101,3,300,326,59-700,533,401,59700,1378,0.520408,1.050,2.441852,-2.441852
3,20101,4,363,329,59-700,552,400,59700,1378,0.714286,1.025,-0.695904,0.695904
4,20101,5,326,304,59-700,519,375,59700,1378,0.377551,0.400,0.000000,0.000000


In [10]:
methods.check_k_thresholds(df, cand_A, cand_B, "_k_score_2")

k > 2.0
trzaskowski: 3795
duda: 1863
---
k > 2.5
trzaskowski: 2938
duda: 1235
---
k > 3.0
trzaskowski: 2358
duda: 865
---


### 3. Komisje, w których kand A uzyskał więcej głosów niż kand B w drugiej turze, mimo że mediana wyników w grupie wskazywała na przewagę kand B;

In [11]:
df = methods.add_anomaly_3(df, cand_A, cand_B, new_col_name="flip")

In [12]:
# na korzyść kandydata A, czyli większą medianę miał B, a więcej głosów dostał A.
cand_A, sum(df['higher_median_' + cand_B] & (df[cand_A + '_r2'] > df[cand_B + '_r2']))

('trzaskowski', 1885)

vs 2025:

('trzaskowski', 2608)

In [13]:
cand_B, sum(df['higher_median_' + cand_A] & (df[cand_B + '_r2'] > df[cand_A + '_r2']))

('duda', 1999)

vs 2025:

('nawrocki', 1843)

In [14]:
df = df.drop(
    columns=[
        f"higher_median_{cand_A}",
        f"higher_median_{cand_B}",
    ]
)

**WYNIKI**:

W grupach, w których większą medianę miał Duda, było 1999 (2025 - 1843) komisji, w których wyższy wynik uzyskał Trzaskowski.

W grupach, w których większą medianę miał Trzaskowski, było 1885 (2025 - 2608) komisji, w których wyższy wyniki uzyskał Nawrocki.

### 4. Kandydat otrzymał mniej głosów w drugiej turze niż w pierwszej

In [15]:
cand_A, sum(df[cand_A + '_r2']<df[cand_A + '_r1'])

('trzaskowski', 106)

W 106 komisjach Trzaskowski uzyskał mniej głosów w drugiej turze niż w pierwszej.

vs 2025:

('trzaskowski', 128)

In [16]:
cand_B, sum(df[cand_B + '_r2']<df[cand_B + '_r1'])

('duda', 284)

W 284 (2025 - 112) komisjach Duda uzyskał mniej głosów w drugiej turze niż w pierwszej.

vs 2025:

('nawrocki', 112)


Przykładowe anomalie na korzyść Trzaskowskiego:

In [17]:
df[df[cand_B + '_r2'] < df[cand_B + '_r1']].sort_values(by=cand_B + '_r1', ascending=False).head()

,teryt_gmina,polling_station_id,trzaskowski_r1,duda_r1,postal_code,trzaskowski_r2,duda_r2,postal_clean,bucket,trzaskowski_k_score_1,duda_k_score_1,trzaskowski_k_score_2,duda_k_score_2,trzaskowski_flip,duda_flip
10728,140611,6,287,684,05-660,842,506,05660,94,5.273913,0.258462,4.060803,-4.060803,True,False
21288,247101,16,160,493,41-940,483,447,41940,1081,0.000000,0.000000,14.427480,-14.427480,False,False
5955,81006,1,125,402,67-312,488,220,67312,2377,6.472222,-1.346405,14.371655,-14.371655,True,False
24593,301302,1,90,360,64-111,289,339,64111,1497,-0.303448,-1.000000,10.039896,-10.039896,False,False
13392,146507,430,576,307,04-173,854,299,04173,27,0.000000,-3.648649,6.933126,-6.933126,False,False


In [18]:
# Anomalie na korzysc
df[cand_A + '_more_votes'] = df[cand_B + '_r2']<df[cand_B + '_r1']
df[cand_B + '_more_votes'] = df[cand_A + '_r2']<df[cand_A + '_r1']

## Wyniki

In [19]:
# w zależności od wielkości k
k = 2

# deduplikacja, czyli jak wystąpiła więcej niz jedna anomalia, to liczymy taką komisję raz

### Na korzyść Dudy

In [20]:
cand = "duda"
opponent = "trzaskowski"
outliers_df = methods.generate_candidate_outliers(df, cand, opponent, k)

### Liczba występujących anomalii. Czyli w ilu komisjach wystąpiła np. anomalia "pop outlier"

In [21]:
bool_cols = ["pop_outlier", "growth_outlier", "flip", "more_votes"]
print(outliers_df[bool_cols].sum())

pop_outlier       2893
growth_outlier    1863
flip              1999
more_votes         106
dtype: int64


### Liczba komisji, w których wystąpiło minimum N anomalii, czyli minimum 1 (6243) albo wszystkie cztery anomalie (0)

In [22]:
true_counts = outliers_df[bool_cols].sum(axis=1)

# Convert to plain ints
count_distribution = {
    ">=1": int((true_counts >= 1).sum()),
    ">=2": int((true_counts >= 2).sum()),
    ">=3": int((true_counts >= 3).sum()),
    ">=4": int((true_counts >= 4).sum())
}

# Print row by row
for key, value in count_distribution.items():
    print(f"{key}: {value}")

>=1: 6243
>=2: 571
>=3: 47
>=4: 0


### Próba odwzorowania tabelki z wynikami

In [26]:
# suma głosów. jeżeli w komisji występuje więcej niż jedna anomalia, sumuj jej głosy tylko raz
outliers_df["anomalies"] = outliers_df.apply(methods.assign_top_anomaly, axis=1)

In [27]:
outliers_df[cand + "_median_r2"] = outliers_df.groupby("bucket")[cand + "_r2"].transform("median")
outliers_df[opponent + "_median_r2"] = outliers_df.groupby("bucket")[opponent + "_r2"].transform("median")

In [28]:
outliers_df = methods.add_median_corrected_votes(outliers_df, cand, opponent)

In [29]:
summary_df = methods.summarize_by_anomaly(outliers_df, cand, opponent)
summary_df

,flaga,liczba,głosy duda,głosy trzaskowski,różnica przed,głosy duda po,głosy trzaskowski po,różnica po,zmiana
0,pop_outlier,2601,1669922,1575104,94818,1723726,1521300,202426,107608
1,growth_outlier,1575,498375,626723,-128348,548807,576291,-27484,100864
2,flip,1961,559777,445101,114676,454489,550389,-95900,-210576
3,more_votes,106,4442,930,3512,3285,2087,1198,-2314
4,łącznie,6243,2732516,2647858,84658,2730307,2650067,80240,-4418


### Na korzyść Trzaskowskiego

In [ ]:
cand = "trzaskowski"
opponent = "duda"
outliers_df = methods.generate_candidate_outliers(df, cand, opponent, k)

### Liczba występujących anomalii. Czyli w ilu komisjach wystąpiła np. anomalia "pop outlier"

In [ ]:
bool_cols = ["pop_outlier", "growth_outlier", "flip", "more_votes"]
print(outliers_df[bool_cols].sum())

pop_outlier       3430
growth_outlier    3795
flip              1885
more_votes         284
dtype: int64


### Liczba komisji, w których wystąpiło minimum N anomalii, czyli minimum 1 (8406) albo wszystkie cztery anomalie (2)

In [ ]:
true_counts = outliers_df[bool_cols].sum(axis=1)

# Convert to plain ints
count_distribution = {
    ">=1": int((true_counts >= 1).sum()),
    ">=2": int((true_counts >= 2).sum()),
    ">=3": int((true_counts >= 3).sum()),
    ">=4": int((true_counts >= 4).sum())
}

# Print row by row
for key, value in count_distribution.items():
    print(f"{key}: {value}")

>=1: 8406
>=2: 962
>=3: 24
>=4: 2


In [ ]:
outliers_df["anomalies"] = outliers_df.apply(methods.assign_top_anomaly, axis=1)

In [ ]:
outliers_df[cand + "_median_r2"] = outliers_df.groupby("bucket")[cand + "_r2"].transform("median")
outliers_df[opponent + "_median_r2"] = outliers_df.groupby("bucket")[opponent + "_r2"].transform("median")

In [ ]:
outliers_df = methods.add_median_corrected_votes(outliers_df, cand, opponent)

In [ ]:
summary_df = methods.summarize_by_anomaly(outliers_df, cand, opponent)
summary_df

,flaga,liczba,głosy trzaskowski,głosy duda,różnica przed,głosy trzaskowski po,głosy duda po,różnica po,zmiana
0,pop_outlier,3430,2236448,1947792,288656,1919195,2265045,-345850,-634506
1,growth_outlier,3795,778417,1119117,-340700,868343,1029191,-160848,179852
2,flip,1885,841276,692545,148731,684128,849693,-165565,-314296
3,more_votes,284,15017,11279,3738,14449,11847,2602,-1136
